# 0.0 Initial

## 0.1. Import

In [ ]:
!pip uninstall pandas

In [ ]:
!pip install --upgrade pandas

In [ ]:
import pandas            as pd
import numpy             as np
import seaborn           as sns
import matplotlib.pyplot as plt
import xgboost           as xgb

from sklearn    import decomposition   as d
from matplotlib import pyplot          as plt
from sklearn    import model_selection as ms
from keras      import models          as ml
from keras      import layers          as l

from boruta                import BorutaPy
from keras.callbacks       import EarlyStopping
from sklearn.ensemble      import RandomForestRegressor
from sklearn.linear_model  import LinearRegression,Lasso
from numpy                 import asarray
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from IPython.display       import Image

## 0.2. Helper Functions

In [ ]:
def bmi (height, weight):
    bmi = weight/((height/100)**2)
    return bmi

def mean_absolute_error (y,yhat):
    return np.mean(np.abs(y-yhat))

def mean_absolute_percentage_error (y,yhat):
    return np.mean(np.abs(y-yhat)/y)

def mean_squared_error (y,yhat):
    return np.mean(pow((yhat-y),2))

def ml_error (model_name, y, yhat):
    mae = mean_absolute_error(y,yhat)
    mape = mean_absolute_percentage_error(y,yhat)
    rmse = np.sqrt(mean_squared_error(y,yhat))
    
    return pd.DataFrame({'Model Name': model_name,
                        'MAE': mae,
                        'MAPE': mape,
                        'RMSE': rmse}, index=[0])

def perc (x):
    #calculate % people who had CVD
    x['%_cardio_1'] = ""
    j = len(x)
    for i in range(len(x)):
        i=i+1
        if i < j:
            p = ((x['total'][i])/(x['total'][i-1]+x['total'][i]))*100
            x['%_cardio_1'][i] = p
        else:
            exit

## 0.3. Loading Data

In [ ]:
df_raw = pd.read_csv('cardio_train.csv', low_memory=False)

In [ ]:
df_raw

# 1.0. Data Description

In [ ]:
df1 = df_raw.copy()

## 1.1. Data Dimensions

In [ ]:
print('Number of columns:', format(df1.shape[1]))
print('Number of rows:', format(df1.shape[0]))

## 1.2. Data Types

In [ ]:
df1.dtypes

## 1.3. Check NA

In [ ]:
df1.isna().sum()

## 1.4. Unit change

In [4]:
#age (int)
df1['age'] = (df1['age']/365).astype(int)

#weight (int)
df1['weight'] = df1['weight'].astype(int)

df1.dtypes

NameError: name 'df1' is not defined

## 1.5. Descriptive Statiscal

In [ ]:
#central tendency - mean, median
ct1 = pd.DataFrame(df1.apply(np.mean)).T
ct2 = pd.DataFrame(df1.apply(np.median)).T

#Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame(df1.apply(min)).T
d2 = pd.DataFrame(df1.apply(max)).T
d3 = pd.DataFrame(df1.apply(lambda x: x.max() - x.min())).T
d4 = pd.DataFrame(df1.apply(np.std)).T
d5 = pd.DataFrame(df1.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(df1.apply(lambda x: x.kurtosis())).T

#concatenate
m = pd.concat([d1,d2,d3,ct1,ct2,d4,d5,d6]).T.reset_index()
m.columns = ['attributes','min','max','range','mean','median','std','skew','kurtosis']
m

# 2.0. Feature Engineering

In [ ]:
df2 = df1.copy()

**Identification**

Age | Objective Feature | age | int (days)

Height | Objective Feature | height | int (cm) |

Weight | Objective Feature | weight | float (kg) |

Gender | Objective Feature | gender | categorical code | 1 - women | 2 - men



**Review of Systems**

Systolic blood pressure | Examination Feature | ap_hi | int |

Diastolic blood pressure | Examination Feature | ap_lo | int |

Cholesterol | Examination Feature | cholesterol | 1: normal, 2: above normal, 3: well above normal |

Glucose | Examination Feature | gluc | 1: normal, 2: above normal, 3: well above normal |

**Habits**

Smoking | Subjective Feature | smoke | binary |

Alcohol intake | Subjective Feature | alco | binary |

Physical activity | Subjective Feature | active | binary |

**Target Variable**

Presence or absence of cardiovascular disease | Target Variable | cardio | binary |

## 2.1 Mind Map Hypothesis

In [ ]:
Image('CCD.png')

## 2.2 Hypothesis Creation

### 2.1.1 Identification Hypothesis

**1.** People over 50 are 20% more likely to have CVDs

**2.** People shorter than 170 cm (5'7'') have a 5% chance of CVDs

**3.** Men are 30% more likely than women to develop a CVD

**4.** People over 100 kg (220 lb) have a 40% chance of having a CVD

**5.** People with a BMI above 30 (obese class I) have a 25% chance of having a CVD

### 2.1.2 Habits Hypothesis

**1.** People who do not engage in physical activity are 40% more likely to have CDCs

**2.** People who regularly consume alcohol have a 10% chance of having CVD

**3.** People who smoke are 20% more likely to have CVD

### 2.1.3 Exams Hypothesis

**1.** People who have well above normal cholesterol have a 50% chance of having CVD

**2.** People who have well above normal glucose have a 20% chance of having CVD

**3.** People who have high diastolic blood pressure have a 40% chance of having CVD

**4.** People who have high systolic blood pressure have a 40% chance of having CVD

## 2.3. Final Hypothesis

**1.** People over 50 are 20% more likely to have CVDs

**2.** People shorter than 170 cm (5'7'') have a 5% chance of CVDs

**3.** Men are 30% more likely than women to develop a CVD

**4.** People over 100 kg (220 lb) have a 40% chance of having a CVD

**5.** People with a BMI above 30 (obese class I) have a 25% chance of having a CVD

**6.** People who do not engage in physical activity are 40% more likely to have CVDs

**7.** People who regularly consume alcohol have a 10% chance of having CVD

**8.** People who smoke are 20% more likely to have CVD

**9.** People who have well above normal cholesterol have a 50% chance of having CVD

**10.** People who have well above normal glucose have a 20% chance of having CVD

**11.** People who have high blood pressure have a 40% chance of having CVD

## 2.4. Feature Engineering

In [ ]:
#BMI
df2['bmi'] = bmi(df2['height'], df2['weight'])

In [ ]:
#blood pressure classification
df2['blood_pressure'] = ''

df2['blood_pressure'] = df2.apply(lambda x: '0' if (x['ap_hi'] > 0 and x['ap_hi'] < 120) and (x['ap_lo'] > 0 and x['ap_lo'] < 80) else '' 
                                  '1' if (x['ap_hi'] >= 120 and x['ap_hi'] <= 129) and (x['ap_lo'] < 80) else ''
                                  '2' if (x['ap_hi'] >= 130 and x['ap_hi'] <= 139) or (x['ap_lo'] >= 80 and x['ap_lo'] <= 89) else ''
                                  '3' if (x['ap_hi'] >= 140 and x['ap_hi'] <= 179) or (x['ap_lo'] >= 90 and x['ap_lo'] <= 119) else ''
                                  '4' if x['ap_hi'] >= 180 or x['ap_lo'] >= 120 else '', axis=1)

df2['blood_pressure'] = pd.to_numeric(df2['blood_pressure'],errors = 'coerce')

# 3.0. Data Filtering

In [ ]:
df3 = df2.copy()

In [ ]:
#blood pressure
df3 = df3[~df3['blood_pressure'].isna()]

#height
df3 = df3[(df3['height'] >= 140) & (df3['height'] <= 200)]

#weight
df3 = df3[(df3['weight'] >= 50) & (df3['weight'] <= 150)]

#ap_hi
df3 = df3[(df3['ap_hi'] >= 0) & (df3['ap_hi'] <= 250)]

#ap_lo
df3 = df3[(df3['ap_lo'] >= 0) & (df3['ap_lo'] <= 250)]

In [ ]:
#central tendency - mean, median
ct1 = pd.DataFrame(df3.apply(np.mean)).T
ct2 = pd.DataFrame(df3.apply(np.median)).T

#Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame(df3.apply(min)).T
d2 = pd.DataFrame(df3.apply(max)).T
d3 = pd.DataFrame(df3.apply(lambda x: x.max() - x.min())).T
d4 = pd.DataFrame(df3.apply(np.std)).T
d5 = pd.DataFrame(df3.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(df3.apply(lambda x: x.kurtosis())).T
d7 = pd.DataFrame(df3.apply(lambda x: len(x))).T

#concatenate
m = pd.concat([d1,d2,d3,ct1,ct2,d4,d5,d6,d7]).T.reset_index()
m.columns = ['attributes','min','max','range','mean','median','std','skew','kurtosis','total']
m

# 4.0. Exploratory Data Analysis

In [ ]:
df4 = df3.copy()

## 4.1 Univariate Analysis

### 4.1.1 Response variable

In [ ]:
#sns.displot(df4['cardio'], kde=True)

### 4.1.2 Numeric variable

In [ ]:
df42 = df4.drop('id', axis=1)
df42.hist(figsize=(24,12), bins=50);

## 4.2. Bivariate Analysis

**1. Between 39 - 64. People over 50 are on average 20% more likely to have CVD**

**TRUE** On average people over 50 are 21% more likely to have CVD. And 63 years old has the highest incidence.

In [ ]:
aux1 = df4[['age', 'cardio']].groupby('age').sum().reset_index().rename(columns={0:'total'})

fig_dims = (15, 7)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(x='age', y='cardio', ax=ax, data=aux1)

In [ ]:
aux1 = df4[(df4['age'] >= 39) & (df4['cardio'] >= 1)]
aux1 = aux1[['age', 'cardio']].groupby('age').size().reset_index().rename(columns={0:'total_1'})
aux1

aux2 = df4[(df4['age'] >= 39) & (df4['cardio'] <= 0)]
aux2 = aux2[['age', 'cardio']].groupby('age').size().reset_index().rename(columns={0:'total_0'})
aux2

fig = plt.figure(figsize=(20,15))

ax = fig.add_subplot(111) #create matplotlib axes

width = 0.2

aux1.plot(x='age', y='total_1', kind='bar', color='red', ax=ax, width=width, position=0)
aux2.plot(x='age', y='total_0', kind='bar', color='blue', ax=ax, width=width, position=1)
plt.show()

In [ ]:
aux1 = df4[['age', 'cardio']].groupby(['age','cardio']).size().reset_index().rename(columns={0:'total'})

In [ ]:
#calculate % people who had CVD
aux1['%_cardio_1'] = ""

j = len(aux1)
for i in range(len(aux1)):
    i=i+1
    if i < j:
        perc = ((aux1['total'][i])/(aux1['total'][i-1]+aux1['total'][i]))*100
        aux1['%_cardio_1'][i] = perc
    else:
        exit

In [ ]:
#aux1 = perc(aux1)
aux1 = aux1[(aux1['cardio'] >=1)]
aux_50 = aux1[(aux1['age'] >= 50)]
aux_50 = pd.DataFrame(aux_50.apply(np.mean)).T
aux_50

aux_49 = aux1[(aux1['age'] < 50)]
aux_49 = pd.DataFrame(aux_49.apply(np.mean)).T
aux_49

m = pd.concat([aux_50, aux_49]).T.reset_index()
m.columns = ['index','over 50','below 50']
m

In [ ]:
fig_dims = (18, 10)
fig, ax = plt.subplots(figsize=fig_dims)

graph = sns.barplot(x='age', y='%_cardio_1', ax=ax, data=aux1)
graph.axvline(10.5, ymin=0.0, ymax=1.0, linestyle ="--", color='blue')
graph.axhline(51, xmin=0.0, xmax=1.0, linestyle ="--", color='red')
plt.title('% People who had CVD per age')
plt.show()

In [ ]:
#boxplot age+cardio
sns.boxplot(x='age', data=aux1)
ax.set(xlabel='common xlabel', ylabel='common ylabel')
plt.show()

**2. Between 135 - 193. People shorter than 170 cm (5'7'') have a 5% chance of CVDs**

**FALSE** On average people shorter than 170 cm have 1% more likely to have CVD.

In [ ]:
#histogram
aux1 = df4[(df4['height'] >= 140) & (df4['height'] <= 180)]
aux1 = aux1[['height', 'cardio']].groupby('height').sum().reset_index()

fig_dims = (18, 10)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(x='height', y='cardio', ax=ax, data=aux1)

In [ ]:
#histogram with and without CVD
aux1 = df4[(df4['cardio'] >= 1) & (df4['height'] >= 140) & (df4['height'] <= 190)]
aux1 = aux1[['height', 'cardio']].groupby('height').size().reset_index().rename(columns={0:'total_1'})
aux1

aux2 = df4[(df4['cardio'] <= 0) & (df4['height'] >= 140) & (df4['height'] <= 190)]
aux2 = aux2[['height', 'cardio']].groupby('height').size().reset_index().rename(columns={0:'total_0'})
aux2

fig = plt.figure(figsize=(20,15))

ax = fig.add_subplot(111)

width = 0.2

aux1.plot(x='height', y='total_1', kind='bar', color='red', ax=ax, width=width, position=0)
aux2.plot(x='height', y='total_0', kind='bar', color='blue', ax=ax, width=width, position=1)
plt.show()

In [ ]:
aux1 = df4[(df4['height'] >= 135) & (df4['height'] <= 193)]
aux1 = aux1[['height', 'cardio']].groupby(['height','cardio']).size().reset_index().rename(columns={0:'total'})
#aux1

In [ ]:
#calculate % people who had CVD
aux1['%_cardio_1'] = ""

j = len(aux1)
for i in range(len(aux1)):
    i=i+1
    if i < j:
        perc = ((aux1['total'][i])/(aux1['total'][i-1]+aux1['total'][i]))*100
        aux1['%_cardio_1'][i] = perc
    else:
        exit

In [ ]:
#aux1 = aux1[(aux1['cardio'] >= 1)]
#aux1

In [ ]:
#average on CVD
aux1 = aux1[(aux1['cardio'] >= 1)]
aux_170 = aux1[(aux1['height'] >= 170)]
aux_170 = pd.DataFrame(aux_170.apply(np.mean)).T
aux_170

aux_169 = aux1[(aux1['height'] < 170)]
aux_169 = pd.DataFrame(aux_169.apply(np.mean)).T
aux_169

m = pd.concat([aux_170, aux_169]).T.reset_index()
m.columns = ['index','over 170','below 169']
m

In [ ]:
fig_dims = (18, 10)
fig, ax = plt.subplots(figsize=fig_dims)
#plt.axvline(29.5, 0.0,1.0)
graph = sns.barplot(x='height', y='%_cardio_1', ax=ax, data=aux1)
graph.axvline(29.5, ymin=0.0, ymax=1.0, linestyle ="--", color='blue')
graph.axhline(51, xmin=0.0, xmax=1.0, linestyle ="--", color='red')
plt.title('% People who had CVD per height')
plt.show()

In [ ]:
#boxplot height+cardio
#aux1 = df3[['height', 'cardio']].groupby('height').sum().reset_index()
#sns.boxplot(x='height', data=aux1)

**3. Men are 30% more likely than women to develop a CVD**

**FALSE** Men had 50,5% of cases of CVD, while women had 49,7%

In [ ]:
#transform to categorical
df42 = df4.copy()
df42['gender'] = df42['gender'].apply({1:'women', 2:'men'}.get)

In [ ]:
aux1 = df42[(df42['cardio'] >= 1)]
aux1 = aux1[['gender', 'cardio']].groupby('gender').size().reset_index().rename(columns={0:'total_1'})
aux1

aux2 = df42[(df42['cardio'] <= 0)]
aux2 = aux2[['gender', 'cardio']].groupby('gender').size().reset_index().rename(columns={0:'total_0'})
aux2

fig = plt.figure(figsize=(10,10))

ax = fig.add_subplot(111)

width = 0.2

aux1.plot(x='gender', y='total_1', kind='bar', color='red', ax=ax, width=width, position=0)
aux2.plot(x='gender', y='total_0', kind='bar', color='blue', ax=ax, width=width, position=1)
plt.show()

In [ ]:
aux1 = df42[['gender', 'cardio']].groupby(['gender','cardio']).size().reset_index().rename(columns={0:'total'})
aux1

In [ ]:
#calculate % people who had CVD
aux1['%_cardio_1'] = ""

j = len(aux1)
for i in range(len(aux1)):
    i=i+1
    if i < j:
        perc = ((aux1['total'][i])/(aux1['total'][i-1]+aux1['total'][i]))*100
        aux1['%_cardio_1'][i] = perc
    else:
        exit

In [5]:
aux1 = aux1[(aux1['cardio'] >= 1)]

NameError: name 'aux1' is not defined

In [ ]:
fig_dims = (6, 8)
fig, ax = plt.subplots(figsize=fig_dims)
#plt.axvline(29.5, 0.0,1.0)
graph = sns.barplot(x='gender', y='%_cardio_1', ax=ax, data=aux1)
#graph.axvline(29.5, ymin=0.0, ymax=1.0, linestyle ="--", color='blue')
graph.axhline(51, xmin=0.0, xmax=1.0, linestyle ="--", color='red')
plt.title('% People who had CVD per active')
plt.show()

In [ ]:
aux1

**4. Between 33 - 130. People over 100 kg (220 lb) have a 40% chance of having a CVD**

**FALSE** On average people with over 100 kg have 25% more chance of having a CVD

In [ ]:
aux1 = df4[(df4['weight'] >= 5) & (df4['weight'] <= 55)]
aux1 = aux1[['weight', 'cardio']].groupby('weight').sum().reset_index()

fig_dims = (18, 10)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(x='weight', y='cardio', ax=ax, data=aux1)

In [ ]:
aux1 = df4[(df4['weight'] >= 33) & (df4['weight'] <= 100) & (df4['cardio'] >= 1)]
aux1 = aux1[['weight', 'cardio']].groupby('weight').size().reset_index().rename(columns={0:'total_1'})
aux1

aux2 = df4[(df4['weight'] >= 33) & (df4['weight'] <= 100) & (df4['cardio'] <= 0)]
aux2 = aux2[['weight', 'cardio']].groupby('weight').size().reset_index().rename(columns={0:'total_0'})
aux2

fig = plt.figure(figsize=(20,15))

ax = fig.add_subplot(111)

width = 0.2

aux1.plot(x='weight', y='total_1', kind='bar', color='red', ax=ax, width=width, position=0)
aux2.plot(x='weight', y='total_0', kind='bar', color='blue', ax=ax, width=width, position=1)
plt.show()

In [ ]:
aux1 = df4[(df4['weight'] >= 33) & (df4['weight'] <= 130)]
aux1 = aux1[['weight', 'cardio']].groupby(['weight','cardio']).size().reset_index().rename(columns={0:'total'})
#aux1

In [ ]:
#calculate % people who had CVD
aux1['%_cardio_1'] = ""

j = len(aux1)
for i in range(len(aux1)):
    i=i+1
    if i < j:
        perc = ((aux1['total'][i])/(aux1['total'][i-1]+aux1['total'][i]))*100
        aux1['%_cardio_1'][i] = perc
    else:
        exit

In [ ]:
aux1 = aux1[(aux1['cardio'] >=1)]
aux_100 = aux1[(aux1['weight'] >= 100)]
aux_100 = pd.DataFrame(aux_100.apply(np.mean)).T

aux_99 = aux1[(aux1['weight'] < 100)]
aux_99 = pd.DataFrame(aux_99.apply(np.mean)).T

m = pd.concat([aux_100, aux_99]).T.reset_index()
m.columns = ['index','over 100','below 100']
m

In [ ]:
fig_dims = (20,15)
fig, ax = plt.subplots(figsize=fig_dims)
#plt.axvline(29.5, 0.0,1.0)
graph = sns.barplot(x='weight', y='%_cardio_1', ax=ax, data=aux1)
graph.axvline(14.5, ymin=0.0, ymax=1.0, linestyle ="--", color='blue')
graph.axhline(51, xmin=0.0, xmax=1.0, linestyle ="--", color='red')
plt.title('% People who had CVD per BMI')
plt.show()

In [ ]:
#boxplot weight+cardio
#aux1 = df3[['weight', 'cardio']].groupby('weight').sum().reset_index()
#sns.boxplot(x='weight', data=aux1)

**5. People with a BMI above 30 (obese class I) have a 25% chance of having a CVD**

**FALSE** People with a BMI less than 30 have 72% chance of having CVD

In [ ]:
#convert type bmi (int)
df4['bmi'] = df4['bmi'].astype(int)

In [ ]:
aux1 = df4[(df4['bmi'] >= 15) & (df4['bmi'] <= 50) & (df4['cardio'] >= 1)]
aux1 = aux1[['bmi', 'cardio']].groupby('bmi').size().reset_index().rename(columns={0:'total_1'})
aux1

aux2 = df4[(df4['bmi'] >= 15) & (df4['bmi'] <= 50) & (df4['cardio'] <= 0)]
aux2 = aux2[['bmi', 'cardio']].groupby('bmi').size().reset_index().rename(columns={0:'total_0'})
aux2

fig = plt.figure(figsize=(20,15))

ax = fig.add_subplot(111) # Create matplotlib axes
#ax2 = ax.twinx()

width = 0.2

#aux1.set_index(['cardio','bmi']).plot.bar(figsize=(20,10))
aux1.plot(x='bmi', y='total_1', kind='bar', color='red', ax=ax, width=width, position=0)
aux2.plot(x='bmi', y='total_0', kind='bar', color='blue', ax=ax, width=width, position=1)
plt.show()
#fig_dims = (18, 10)
#fig, ax = plt.subplots(figsize=fig_dims)
#sns.barplot(x='bmi', y='total', ax=ax, data=aux1)

In [ ]:
aux1 = df4[(df4['bmi'] >= 15) & (df4['bmi'] <= 50)]
aux1 = aux1[['bmi', 'cardio']].groupby(['bmi','cardio']).size().reset_index().rename(columns={0:'total'})
#aux1

In [ ]:
#calculate % people who had CVD
aux1['%_cardio_1'] = ""

j = len(aux1)
for i in range(len(aux1)):
    i=i+1
    if i < j:
        perc = ((aux1['total'][i])/(aux1['total'][i-1]+aux1['total'][i]))*100
        aux1['%_cardio_1'][i] = perc
    else:
        exit

In [ ]:
aux1 = aux1[(aux1['cardio'] >=1)]
aux_30 = aux1[(aux1['bmi'] >= 30)]
aux_30 = pd.DataFrame(aux_30.apply(np.mean)).T
aux_30

aux_29 = aux1[(aux1['bmi'] < 30)]
aux_29 = pd.DataFrame(aux_29.apply(np.mean)).T
aux_29

m = pd.concat([aux_30, aux_29]).T.reset_index()
m.columns = ['index','over 30','above 30']
m

In [ ]:
fig_dims = (18, 10)
fig, ax = plt.subplots(figsize=fig_dims)
#plt.axvline(29.5, 0.0,1.0)
graph = sns.barplot(x='bmi', y='%_cardio_1', ax=ax, data=aux1)
graph.axvline(14.5, ymin=0.0, ymax=1.0, linestyle ="--", color='blue')
graph.axhline(51, xmin=0.0, xmax=1.0, linestyle ="--", color='red')
plt.title('% People who had CVD per BMI')
plt.show()

In [ ]:
#boxplot bmi+cardio
#aux1 = df3[['bmi', 'cardio']].groupby('bmi').sum().reset_index()
#sns.boxplot(x='bmi', data=aux1)

**6. People who do not engage in physical activity are 40% more likely to have CVDs**

In [ ]:
#transform to categorical
df42 = df4.copy()
df42['active'] = df42['active'].apply({0:'no', 1:'yes'}.get)

In [ ]:
aux1 = df42[(df42['cardio'] >= 1)]
aux1 = aux1[['active', 'cardio']].groupby('active').size().reset_index().rename(columns={0:'total_1'})
aux1

aux2 = df42[(df42['cardio'] <= 0)]
aux2 = aux2[['active', 'cardio']].groupby('active').size().reset_index().rename(columns={0:'total_0'})
aux2

fig = plt.figure(figsize=(20,15))

ax = fig.add_subplot(111)

width = 0.2

aux1.plot(x='active', y='total_1', kind='bar', color='red', ax=ax, width=width, position=0)
aux2.plot(x='active', y='total_0', kind='bar', color='blue', ax=ax, width=width, position=1)
plt.show()

In [ ]:
aux1 = df4[['active', 'cardio']].groupby(['active','cardio']).size().reset_index().rename(columns={0:'total'})
aux1

In [ ]:
#calculate % people who had CVD
aux1['%_cardio_1'] = ""

j = len(aux1)
for i in range(len(aux1)):
    i=i+1
    if i < j:
        perc = ((aux1['total'][i])/(aux1['total'][i-1]+aux1['total'][i]))*100
        aux1['%_cardio_1'][i] = perc
    else:
        exit

In [ ]:
aux1 = aux1[(aux1['cardio'] >= 1)]

In [ ]:
fig_dims = (6, 8)
fig, ax = plt.subplots(figsize=fig_dims)
#plt.axvline(29.5, 0.0,1.0)
graph = sns.barplot(x='active', y='%_cardio_1', ax=ax, data=aux1)
#graph.axvline(29.5, ymin=0.0, ymax=1.0, linestyle ="--", color='blue')
graph.axhline(51, xmin=0.0, xmax=1.0, linestyle ="--", color='red')
plt.title('% People who had CVD per active')
plt.show()

In [ ]:
aux1

**7. People who regularly consume alcohol have a 10% chance of having CVD**

In [ ]:
#transform to categorical
df42 = df4.copy()
df42['alco'] = df42['alco'].apply({0:'no', 1:'yes'}.get)

In [ ]:
aux1 = df42[(df42['cardio'] >= 1)]
aux1 = aux1[['alco', 'cardio']].groupby('alco').size().reset_index().rename(columns={0:'total_1'})
aux1

aux2 = df42[(df42['cardio'] <= 0)]
aux2 = aux2[['alco', 'cardio']].groupby('alco').size().reset_index().rename(columns={0:'total_0'})
aux2

fig = plt.figure(figsize=(10,10))

ax = fig.add_subplot(111)

width = 0.2

aux1.plot(x='alco', y='total_1', kind='bar', color='red', ax=ax, width=width, position=0)
aux2.plot(x='alco', y='total_0', kind='bar', color='blue', ax=ax, width=width, position=1)
plt.show()

In [ ]:
aux1 = df42[['alco', 'cardio']].groupby(['alco','cardio']).size().reset_index().reset_index().rename(columns={0:'total'})
aux1

In [ ]:
#calculate % people who had CVD
aux1['%_cardio_1'] = ""

j = len(aux1)
for i in range(len(aux1)):
    i=i+1
    if i < j:
        perc = ((aux1['total'][i])/(aux1['total'][i-1]+aux1['total'][i]))*100
        aux1['%_cardio_1'][i] = perc
    else:
        exit

In [ ]:
aux1 = aux1[(aux1['cardio'] >= 1)]

In [ ]:
fig_dims = (6, 8)
fig, ax = plt.subplots(figsize=fig_dims)
#plt.axvline(29.5, 0.0,1.0)
graph = sns.barplot(x='alco', y='%_cardio_1', ax=ax, data=aux1)
#graph.axvline(29.5, ymin=0.0, ymax=1.0, linestyle ="--", color='blue')
graph.axhline(51, xmin=0.0, xmax=1.0, linestyle ="--", color='red')
plt.title('% People who had CVD per alco')
plt.show()

In [ ]:
aux1

**8. People who smoke are 20% more likely to have CVD**

In [ ]:
#transform to categorical
df42 = df4.copy()
df42['smoke'] = df42['smoke'].apply({0:'no', 1:'yes'}.get)

In [ ]:
aux1 = df42[(df42['cardio'] >= 1)]
aux1 = aux1[['smoke', 'cardio']].groupby('smoke').size().reset_index().rename(columns={0:'total_1'})
aux1

aux2 = df42[(df42['cardio'] <= 0)]
aux2 = aux2[['smoke', 'cardio']].groupby('smoke').size().reset_index().rename(columns={0:'total_0'})
aux2

fig = plt.figure(figsize=(10,10))

ax = fig.add_subplot(111)

width = 0.2

aux1.plot(x='smoke', y='total_1', kind='bar', color='red', ax=ax, width=width, position=0)
aux2.plot(x='smoke', y='total_0', kind='bar', color='blue', ax=ax, width=width, position=1)
plt.show()

In [ ]:
aux1 = df42[['smoke', 'cardio']].groupby(['smoke','cardio']).size().reset_index().rename(columns={0:'total'})
aux1

In [ ]:
#calculate % people who had CVD
aux1['%_cardio_1'] = ""

j = len(aux1)
for i in range(len(aux1)):
    i=i+1
    if i < j:
        perc = ((aux1['total'][i])/(aux1['total'][i-1]+aux1['total'][i]))*100
        aux1['%_cardio_1'][i] = perc
    else:
        exit

In [ ]:
aux1 = aux1[(aux1['cardio'] >= 1)]

In [ ]:
fig_dims = (6, 8)
fig, ax = plt.subplots(figsize=fig_dims)
#plt.axvline(29.5, 0.0,1.0)
graph = sns.barplot(x='smoke', y='%_cardio_1', ax=ax, data=aux1)
#graph.axvline(29.5, ymin=0.0, ymax=1.0, linestyle ="--", color='blue')
graph.axhline(51, xmin=0.0, xmax=1.0, linestyle ="--", color='red')
plt.title('% People who had CVD per smoke')
plt.show()

In [ ]:
aux1

**9. People who have well above normal cholesterol have a 50% chance of having CVD**

In [ ]:
#transform to categorical
df42 = df4.copy()
df42['cholesterol'] = df42['cholesterol'].apply({1:'normal', 2:'above normal', 3:'well above normal'}.get)

In [ ]:
aux1 = df42[(df42['cardio'] >= 1)]
aux1 = aux1[['cholesterol', 'cardio']].groupby('cholesterol').size().reset_index().rename(columns={0:'total_1'})
aux1

aux2 = df42[(df42['cardio'] <= 0)]
aux2 = aux2[['cholesterol', 'cardio']].groupby('cholesterol').size().reset_index().rename(columns={0:'total_0'})
aux2

fig = plt.figure(figsize=(10,10))

ax = fig.add_subplot(111)

width = 0.2

aux1.plot(x='cholesterol', y='total_1', kind='bar', color='red', ax=ax, width=width, position=0)
aux2.plot(x='cholesterol', y='total_0', kind='bar', color='blue', ax=ax, width=width, position=1)
plt.show()

In [ ]:
aux1 = df42[['cholesterol', 'cardio']].groupby(['cholesterol','cardio']).size().reset_index().rename(columns={0:'total'})
aux1

In [ ]:
#calculate % people who had CVD
aux1['%_cardio_1'] = ""

j = len(aux1)
for i in range(len(aux1)):
    i=i+1
    if i < j:
        perc = ((aux1['total'][i])/(aux1['total'][i-1]+aux1['total'][i]))*100
        aux1['%_cardio_1'][i] = perc
    else:
        exit

In [ ]:
aux1 = aux1[(aux1['cardio'] >= 1)]

In [ ]:
fig_dims = (6, 8)
fig, ax = plt.subplots(figsize=fig_dims)
#plt.axvline(29.5, 0.0,1.0)
graph = sns.barplot(x='cholesterol', y='%_cardio_1', ax=ax, data=aux1)
#graph.axvline(29.5, ymin=0.0, ymax=1.0, linestyle ="--", color='blue')
graph.axhline(51, xmin=0.0, xmax=1.0, linestyle ="--", color='red')
plt.title('% People who had CVD per cholesterol')
plt.show()

In [ ]:
aux1

**10. People who have well above normal glucose have a 20% chance of having CVD**

In [ ]:
#transform to categorical
df42 = df4.copy()
df42['gluc'] = df42['gluc'].apply({1:'normal', 2:'above normal', 3:'well above normal'}.get)

In [6]:
aux1 = df42[(df42['cardio'] >= 1)]
aux1 = aux1[['gluc', 'cardio']].groupby('gluc').size().reset_index().rename(columns={0:'total_1'})
aux1

aux2 = df42[(df42['cardio'] <= 0)]
aux2 = aux2[['gluc', 'cardio']].groupby('gluc').size().reset_index().rename(columns={0:'total_0'})
aux2

fig = plt.figure(figsize=(10,10))

ax = fig.add_subplot(111)

width = 0.2

aux1.plot(x='gluc', y='total_1', kind='bar', color='red', ax=ax, width=width, position=0)
aux2.plot(x='gluc', y='total_0', kind='bar', color='blue', ax=ax, width=width, position=1)
plt.show()

NameError: name 'df42' is not defined

In [ ]:
aux1 = df42[['gluc', 'cardio']].groupby(['gluc','cardio']).size().reset_index().rename(columns={0:'total'})
aux1

In [ ]:
#calculate % people who had CVD
aux1['%_cardio_1'] = ""

j = len(aux1)
for i in range(len(aux1)):
    i=i+1
    if i < j:
        perc = ((aux1['total'][i])/(aux1['total'][i-1]+aux1['total'][i]))*100
        aux1['%_cardio_1'][i] = perc
    else:
        exit

In [ ]:
aux1 = aux1[(aux1['cardio'] >= 1)]

In [ ]:
fig_dims = (6, 8)
fig, ax = plt.subplots(figsize=fig_dims)
#plt.axvline(29.5, 0.0,1.0)
graph = sns.barplot(x='gluc', y='%_cardio_1', ax=ax, data=aux1)
#graph.axvline(29.5, ymin=0.0, ymax=1.0, linestyle ="--", color='blue')
graph.axhline(51, xmin=0.0, xmax=1.0, linestyle ="--", color='red')
plt.title('% People who had CVD per gluc')
plt.show()

In [ ]:
aux1

**11. People who hypertension have a 40% chance of having CVD**

**TRUE** On average have 50% chance of having a CVD

In [ ]:
#aux1 = df3[(df3['blood_pressure'] == '')]
#aux1

In [ ]:
aux1 = df4[(df4['cardio'] >= 1)]
aux1 = aux1[['blood_pressure', 'cardio']].groupby('blood_pressure').size().reset_index().rename(columns={0:'total_1'})
aux1

aux2 = df4[(df4['cardio'] <= 0)]
aux2 = aux2[['blood_pressure', 'cardio']].groupby('blood_pressure').size().reset_index().rename(columns={0:'total_0'})
aux2

fig = plt.figure(figsize=(10,10))

ax = fig.add_subplot(111)

width = 0.2

aux1.plot(x='blood_pressure', y='total_1', kind='bar', color='red', ax=ax, width=width, position=0)
aux2.plot(x='blood_pressure', y='total_0', kind='bar', color='blue', ax=ax, width=width, position=1)
plt.show()

In [ ]:
aux1 = df4[['blood_pressure', 'cardio']].groupby(['blood_pressure','cardio']).size().reset_index().rename(columns={0:'total'})
aux1

In [ ]:
#calculate % people who had CVD
aux1['%_cardio_1'] = ""

j = len(aux1)
for i in range(len(aux1)):
    i=i+1
    if i < j:
        perc = ((aux1['total'][i])/(aux1['total'][i-1]+aux1['total'][i]))*100
        aux1['%_cardio_1'][i] = perc
    else:
        exit

In [ ]:
aux1 = aux1[(aux1['cardio'] >= 1)]

In [ ]:
fig_dims = (10, 8)
fig, ax = plt.subplots(figsize=fig_dims)
#plt.axvline(29.5, 0.0,1.0)
graph = sns.barplot(x='blood_pressure', y='%_cardio_1', ax=ax, data=aux1)
#graph.axvline(29.5, ymin=0.0, ymax=1.0, linestyle ="--", color='blue')
graph.axhline(51, xmin=0.0, xmax=1.0, linestyle ="--", color='red')
plt.title('% People who had CVD per blood_pressure')
plt.show()

In [ ]:
aux1

## 4.3. Multivariate Analysis

In [ ]:
correlations = df4.corr(method='pearson')
fig_dims = (15, 7)
fig, ax = plt.subplots(figsize=fig_dims)
sns.heatmap(correlations,ax=ax, annot=True);

# 5.0. Data Preparation

In [ ]:
df5 = df4.copy()

## 5.1. Normalization

In [ ]:
# define min max scaler
scaler_mm = MinMaxScaler()
scaler_std = StandardScaler()

# transform weight - STD
df5['weight'] = scaler_mm.fit_transform(df5[['weight']].values)

# transform height - STD
df5['height'] = scaler_mm.fit_transform(df5[['height']].values)

# transform age - MM
df5['age'] = scaler_mm.fit_transform(df5[['age']].values)

# transform bmi - MM
df5['bmi'] = scaler_mm.fit_transform(df5[['bmi']].values)

# transform ap_hi - MM
df5['ap_hi'] = scaler_mm.fit_transform(df5[['ap_hi']].values)

# transform ap_lo - MM
df5['ap_lo'] = scaler_mm.fit_transform(df5[['ap_lo']].values)

df5

# 6.0. Feature Selection

In [ ]:
df6 = df5.copy()

In [ ]:
#drop id, ap_hi, ap_lo
df6 = df6.drop(['id'], axis=1)


X = df6.drop(['cardio'], axis=1)
y = df6['cardio'].copy()

In [ ]:
#split dataset into train and test
x, x_test, y, y_test = ms.train_test_split(X, y, test_size=0.2, random_state=32)
x_train, x_val, y_train, y_val = ms.train_test_split( x, y, test_size=0.2 )

## 6.1. Boruta as feature selector

In [ ]:
#training and test dataset for Boruta
#x_train_n = x_train.values
#y_train_n = y_train.values.ravel()

#define RandomForestRegressor
#rf = RandomForestRegressor(n_jobs=-1)

#define Boruta
#boruta = BorutaPy(rf, n_estimators='auto',verbose=2,random_state=42).fit(x_train_n,y_train_n)

## 6.1.1 Best feature from Boruta

In [ ]:
#cols_selected = boruta.support_.tolist()

#best feature
#x_train_fs = x_train
#cols_selected_boruta = x_train_fs.iloc[:,cols_selected].columns.to_list()
#cols_selected_boruta

#not selected from Boruta
#cols_not_selected_boruta = list(np.setdiff1d(x_train_fs.columns,cols_selected_boruta))
#cols_not_selected_boruta

# 7.0. Machine Learning Modeling

## 7.1. Linear Regression Model

In [ ]:
#model
lr = LinearRegression().fit(x_train,y_train)

#prediction
yhat_lr = lr.predict(x_test)

#performance
lr_result = ml_error('Linear Regression', np.expm1(y_test), np.expm1(yhat_lr))
lr_result

## 6.2. Linear Regression Regularized Model - Lasso

In [ ]:
#model
lrr = Lasso(alpha = 0.2).fit(x_train,y_train)

#prediction
yhat_lrr = lrr.predict(x_test)

#performance
lrr_result = ml_error('Linear Regression Regularized', np.expm1(y_test), np.expm1(yhat_lrr))
lrr_result

## 6.3. Random Forest Regressor

In [ ]:
#model
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42).fit(x_train,y_train)

#prediction
yhat_rf = rf.predict(x_test)

#performance
rf_result = ml_error('Random Forest Regressor', np.expm1(y_test), np.expm1(yhat_rf))
rf_result

## 6.4. XGBoost

In [ ]:
#model
model_xgb = xgb.XGBRegressor(objective = 'reg:squarederror',
                            n_estimators=100,
                            eta=0.01,
                            max_depth=10,
                            subsample=0.7,
                            colsample_bytee=0.9).fit(x_train,y_train)

#prediction
yhat_xgb = model_xgb.predict(x_test)

#performance
xgb_result = ml_error('XGBoost Regressor', np.expm1(y_test), np.expm1(yhat_xgb))
xgb_result

## 6.5. Neural Network - MLP

In [ ]:
x_val.shape

In [ ]:
#model definition
model = ml.Sequential()
model.add(l.Dense(11, input_dim=x_train.shape[1], activation='relu'))
model.add(l.Dense(1, activation='softmax'))

#model compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#train model
es = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10000, callbacks=[es], verbose=1)

In [ ]:
##model evaluate
_, train_acc = model.evaluate(x_train, y_train, verbose=0)
_, val_acc = model.evaluate(x_val, y_val, verbose=0)

print('Train ACC:{} - Val ACC:{}'.format(train_acc, val_acc ))

In [ ]:
#define the number of eigenvectors (autovetor) + eigenvalues (autovalores)
pca = d.PCA().fit(x_train)
k = np.argmax(np.cumsum(pca.explained_variance_ratio_) > 0.99)

#change data to new space
pca = d.PCA(k).fit(x_train)

x_train = pca.transform(x_train)
x_val = pca.transform(x_val)

In [ ]:
#ohe = pp.OneHotEncoder()
#y_train = ohe.fit_transform(y_train.values.reshape(-1, 1)).toarray()
#y_val = ohe.transform(y_val.values.reshape(-1, 1)).toarray()

In [ ]:
#model definition
model = ml.Sequential()
model.add(l.Dense(11, input_dim=x_train.shape[1], activation='relu'))
model.add(l.Dense(1, activation='softmax'))

#model compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#train model
es = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, callbacks=[es], verbose=0)

In [ ]:
##model evaluate
_, train_acc = model.evaluate(x_train, y_train, verbose=0)
_, val_acc = model.evaluate(x_val, y_val, verbose=0)

print('Train ACC:{} - Val ACC:{}'.format(train_acc, val_acc ))